In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()


In [3]:
#Load in necessary earth engine assets, including hansen forest change, WHRC biomass data, and our watershed polygons
WHRC_collection = ee.ImageCollection('projects/wri-datalab/WHRC/global/carbon')
hansen = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")
watersheds = ee.FeatureCollection('projects/wri-datalab/Cities4Forests/Cities4Forests-Watersheds-level5-withCities')

#Add population count image, filter to get 2015 data
population_collection = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")
population_count = population_collection.filterDate('2015-01-01','2015-12-31').first()

#Add area in hectares of waterhsed as well
watersheds = ee.FeatureCollection(watersheds.map(lambda x: x.set({'area_ha':x.geometry().area().multiply(0.0001)})))


In [5]:
#Calculate 30 meter pixel area in hectares
pixel_area = ee.Image.pixelArea().multiply(ee.Image.constant(0.0001))

#Get forest loss binary layer and convert to loss area per pixel for loss in forests (tree cover density>30)
forest_mask = hansen.select('treecover2000').gte(30)
loss_mask = hansen.select('loss').updateMask(forest_mask)
loss_area = hansen.select('loss').multiply(pixel_area)
loss_area_masked = loss_area.updateMask(forest_mask)


#Original WHRC is measured in megagrams biomass/hectare
#Collapse WHRC image collection to one image using Reducer, multiply by pixel area to convert density to total,
#And mask to areas with tree cover loss
WHRC = WHRC_collection.reduce(ee.Reducer.mean())
WHRC_biomass_per_pixel = WHRC.multiply(pixel_area)
WHRC_masked = WHRC_biomass_per_pixel.updateMask(loss_area_masked)

#Save projection and resolution
projection_30m = hansen.projection().getInfo()
crs = projection_30m.get('crs')
crsTransform = projection_30m.get('transform')


def compute_loss_stats(feature):
    '''
    Function to calculate loss statistics for a given feature
    '''
    loss_stats = loss_area_masked.reduceRegion(reducer= ee.Reducer.sum(),
                                        geometry= feature.geometry(),
                                        crs = crs,
                                        crsTransform=crsTransform,
                                        maxPixels= 1e13)
    #print(loss_stats.getInfo())
    feature = ee.Feature(feature.set(loss_stats))
    feature = feature.setGeometry(None)
    return feature

    
def compute_whrc_stats(feature):
    '''
    Function to calculate WHRC biomass statistics for a given feature
    '''
    whrc_stats = WHRC_masked.reduceRegion(reducer= ee.Reducer.sum(),
                                        geometry= feature.geometry(),
                                        crs = crs,
                                        crsTransform=crsTransform,
                                        maxPixels= 1e13)
    feature = ee.Feature(feature.set(whrc_stats))
    feature = feature.setGeometry(None)
    return feature

def compute_population_stats(feature):
    '''
    Function to calculate population count statistics for a given feature
    '''
    whrc_stats = population_count.select('population_count').reduceRegion(reducer= ee.Reducer.sum(),
                                        geometry= feature.geometry(),
                                        crs = crs,
                                        crsTransform=crsTransform,
                                        maxPixels= 1e13)
    feature = ee.Feature(feature.set(whrc_stats))
    feature = feature.setGeometry(None)
    return feature


#Calculate loss statistics for each watershed
loss_stats_collection = ee.FeatureCollection(watersheds.map(compute_loss_stats))

#Calculate WHRC statistics for each watershed
whrc_stats_collection = ee.FeatureCollection(watersheds.map(compute_whrc_stats))

#Calculate population count statistics for each watershed
population_stats_collection = ee.FeatureCollection(watersheds.map(compute_population_stats))


#Export results to google drive
export_loss_stats = ee.batch.Export.table.toDrive(
        collection=loss_stats_collection, 
        description = "Hansen_Loss_Watershed_Stats", 
        fileNamePrefix = 'Hansen_Loss_Watershed_Stats')
export_loss_stats.start()

export_whrc_stats = ee.batch.Export.table.toDrive(
        collection=whrc_stats_collection, 
        description = "Hansen_WHRC_Watershed_Stats", 
        fileNamePrefix = 'Hansen_WHRC_Watershed_Stats')
export_whrc_stats.start()

export_population_stats = ee.batch.Export.table.toDrive(
        collection=population_stats_collection, 
        description = "Hansen_Population_Watershed_Stats", 
        fileNamePrefix = 'Hansen_Population_Watershed_Stats')
export_population_stats.start()




